### Full Evaluating CPM with CEBaB

This note book is based on `cebab_evaluation.ipynb`. It takes all the saved results data from running that script, and aggregate portable results. This notebook is only for sorting saved results. Nothing more.


In [ ]:
import pickle
import pandas as pd

In [ ]:
model_types = ["BERT", "lstm", "RoBERTa", "gpt2"]
mega_results = {}
for model in model_types:
    with open(f'../proxy_training_results/{model}-results/results.pkl', 'rb') as f:
        results_dict = pickle.load(f)
    with open(f'../proxy_training_results/{model}-control-results/results.pkl', 'rb') as f:
        control_results_dict = pickle.load(f)
    for k, v in results_dict.items():
        mega_results[k] = v
    for k, v in control_results_dict.items():
        mega_results[k] = v

In [ ]:
important_keys = [
    "seed", "h_dim", "class_num", 
    "control", "beta", "gemma", 
    "cls_dropout", "enc_dropout", 
    "model_arch"
]
values = []
for k, v in mega_results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df = df.sort_values(by=['class_num'], ascending=True)

In [ ]:
group_keys= [
    "class_num", 
    "control",
    "model_arch"
]
df.groupby(group_keys)[["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"]].mean()